In [14]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from properscoring import crps_ensemble
import os
import sys

In [15]:
# Add the path to the src directory (two levels up)
sys.path.append(os.path.abspath('../../'))
from src.data_processing import filter_predictions, predict_cnbs, add_df_to_db
from src.hydro_utils import convert_mm_to_cms

In [2]:
# Directory where the repository is cloned
path_to_repo = '/Users/ljob/Desktop/'

# Path to data directory
dir = path_to_repo + 'cnbs-predictor/data/'

## Read in PCP data from CFS [Kg]
pcp_data = pd.read_csv(dir+'CFS/CFS_APCP_Basin_Sums_archived.csv',sep=',')

## Read in EVAP data from CFS [Kg]
evap_data = pd.read_csv(dir+'CFS/CFS_EVAP_Basin_Sums_archived.csv',sep=',')

## Read in TMP data from CFS [K]
tmp_data = pd.read_csv(dir+'CFS/CFS_TMP_Basin_Avgs_archived.csv',sep=',')

In [16]:
# Path to saved scalers
x_scaler = dir + 'input/x_scaler.joblib'
y_scaler = dir + 'input/y_scaler.joblib'

# Define directory to models to use along with their paths
models_info = [{'model': 'GP', 'path': dir + 'input/GP_trained_model.joblib'},
               {'model': 'RF', 'path': dir + 'input/RF_trained_model.joblib'},
               {'model': 'LR', 'path': dir + 'input/LR_trained_model.joblib'},
               {'model': 'NN', 'path': dir + 'input/NN_trained_model.joblib'}]

In [ ]:
def format_observed_data(df):
    # Melt the observed data to long format and create the 'year_month' column in a single step
    observed_df = df.melt(id_vars=['Year'], var_name='Month', value_name='Observed')

    # Combine 'Year' and 'Month' into a proper 'year_month' format and convert to datetime in one step
    observed_df['year_month'] = (pd.to_datetime(observed_df['Year'].astype(str) + '-' + observed_df['Month'].str[1:4], format='%Y-%b')).dt.strftime('%Y-%m')
    
    
    # Sort by 'year_month' for proper chronological order
    observed_df = observed_df.sort_values(by='year_month')
    observed_df = observed_df[['year_month', 'Observed']].reset_index(drop=True)

    observed_df.replace(-99990.0, np.nan, inplace=True)
    observed_df = observed_df.dropna()

    return observed_df

In [4]:
def calc_skill_metrics(predictions, observations):

    # Standardizing the observed and predicted values
    scaler = StandardScaler()

    # Fit and transform the data
    pred_scaled = scaler.fit_transform(predictions.values.reshape(-1, 1))
    obs_scaled = scaler.fit_transform(observations.values.reshape(-1, 1))

    # RMSE (Root Mean Squared Error) on standardized data
    rmse = np.sqrt(mean_squared_error(obs_scaled, pred_scaled))
    print(f"RMSE (Standardized): {rmse}")

    # R-squared on standardized data
    r_squared = r2_score(obs_scaled, pred_scaled)
    print(f"R-squared (Standardized): {r_squared}")

    # Bias (Average of prediction - observation) on standardized data
    bias = np.mean(pred_scaled - obs_scaled)
    print(f"Bias (Standardized): {bias}")

    # Variance (Variance of predictions) on standardized data
    variance = np.var(pred_scaled)
    print(f"Variance (Standardized): {variance}")

    # CRPS (Continuous Ranked Probability Score)
    # Assuming predicted is a deterministic point forecast (not distribution)
    # CRPS is more meaningful for probabilistic forecasts. Here we use `crps_ensemble`
    # For simplicity, we use the predicted as the ensemble of one prediction (as a proxy).
    crps = crps_ensemble(obs_scaled, pred_scaled)
    print(f"CRPS (Standardized): {np.mean(crps)}")

In [5]:
# Load the observed data
observed_sup = pd.read_csv(dir + 'glcc/LakeSuperior_MonthlyNetBasinSupply_1900to2025.csv', skiprows=11)
observed_mih = pd.read_csv(dir + 'glcc/LakeMichiganHuron_MonthlyNetBasinSupply_1900to2025.csv', skiprows=11)
observed_eri = pd.read_csv(dir + 'glcc/LakeErie_MonthlyNetBasinSupply_1900to2025.csv', skiprows=11)
observed_ont = pd.read_csv(dir + 'glcc/LakeOntario_MonthlyNetBasinSupply_1900to2025.csv', skiprows=11)               

In [6]:
df_obs_sup = format_observed_data(observed_sup)
df_obs_sup.rename(columns={'Observed': 'superior_cnbs_obs'}, inplace=True)
df_obs_mih = format_observed_data(observed_mih)
df_obs_mih.rename(columns={'Observed': 'michigan-huron_cnbs_obs'}, inplace=True)
df_obs_eri = format_observed_data(observed_eri)
df_obs_eri.rename(columns={'Observed': 'erie_cnbs_obs'}, inplace=True)
df_obs_ont = format_observed_data(observed_ont)
df_obs_ont.rename(columns={'Observed': 'ontario_cnbs_obs'}, inplace=True)

In [7]:
df_obs_merged = pd.merge(df_obs_sup, df_obs_mih, on="year_month", how="outer")
df_obs_merged = pd.merge(df_obs_merged, df_obs_eri, on="year_month", how="outer")
df_obs_merged = pd.merge(df_obs_merged, df_obs_ont, on="year_month", how="outer")

print(df_obs_merged)

     year_month  superior_cnbs_obs  michigan-huron_cnbs_obs  erie_cnbs_obs  \
0       1900-01             -520.0                   -180.0         -150.0   
1       1900-02             1160.0                   4770.0         1770.0   
2       1900-03              -80.0                   1970.0         2420.0   
3       1900-04             3340.0                   6770.0         1410.0   
4       1900-05             3710.0                   4920.0          650.0   
...         ...                ...                      ...            ...   
1497    2024-10            -1140.0                  -3740.0        -1900.0   
1498    2024-11             1160.0                    650.0         -590.0   
1499    2024-12             -490.0                   1170.0         1260.0   
1500    2025-01             -760.0                   -740.0         -100.0   
1501    2025-02             -460.0                   1260.0          470.0   

      ontario_cnbs_obs  
0                770.0  
1            

In [65]:
# Surface area [m2] for each region based on the mask file
sa_eri_lake = 18596386416.712486
sa_eri_land = 81902183517.35063
sa_ont_lake = 15569248531.837788
sa_ont_land = 64465190376.66068
sa_mih_lake = 123626283030.46616
sa_mih_land = 236478938454.17358
sa_sup_lake = 78288645587.81192
sa_sup_land = 123736048741.43369

X = pd.DataFrame({
    'year_month': (pd.to_datetime(pcp_data['year'].astype(str) + '-' + pcp_data['month'].astype(str), format='%Y-%m')).dt.strftime('%Y-%m'),
    'superior_lake_precipitation':  pcp_data['WaterSuperior'] / sa_sup_lake,
    'erie_lake_precipitation': pcp_data['WaterErie'] / sa_eri_lake,
    'ontario_lake_precipitation': pcp_data['WaterOntario'] / sa_ont_lake,
    'michigan-huron_lake_precipitation': (pcp_data['WaterMichigan']+pcp_data['WaterHuron']) / sa_mih_lake,
    'superior_land_precipitation': pcp_data['LandSuperior'] / sa_sup_land,
    'erie_land_precipitation': pcp_data['LandErie'] / sa_eri_land,
    'ontario_land_precipitation': pcp_data['LandOntario'] / sa_ont_land,
    'michigan-huron_land_precipitation': (pcp_data['LandMichigan']+pcp_data['LandHuron']) / sa_mih_land,
    'superior_lake_evaporation': evap_data['WaterSuperior'] / sa_sup_lake,
    'erie_lake_evaporation': evap_data['WaterErie'] / sa_eri_lake,
    'ontario_lake_evaporation': evap_data['WaterOntario'] / sa_ont_lake,
    'michigan-huron_lake_evaporation': (evap_data['WaterMichigan']+evap_data['WaterHuron']) / sa_mih_lake,
    'superior_land_evaporation': evap_data['LandSuperior'] / sa_sup_land,
    'erie_land_evaporation': evap_data['LandErie'] / sa_eri_land,
    'ontario_land_evaporation': evap_data['LandOntario'] / sa_ont_land,
    'michigan-huron_land_evaporation': (evap_data['LandMichigan']+evap_data['LandHuron']) / sa_mih_land,
    'superior_lake_air_temperature': tmp_data['WaterSuperior'],
    'erie_lake_air_temperature': tmp_data['WaterErie'],
    'ontario_lake_air_temperature': tmp_data['WaterOntario'],
    'michigan-huron_lake_air_temperature': (tmp_data['WaterMichigan']+tmp_data['WaterHuron']) / 2,
    'superior_land_air_temperature': tmp_data['LandSuperior'],
    'erie_land_air_temperature': tmp_data['LandErie'],
    'ontario_land_air_temperature': tmp_data['LandOntario'],
    'michigan-huron_land_air_temperature': (tmp_data['LandMichigan']+tmp_data['LandHuron']) / 2
})

X.set_index(['year_month'], inplace=True)

In [66]:
print(X)

            superior_lake_precipitation  erie_lake_precipitation  \
year_month                                                         
2012-01                      104.151411                75.620541   
2012-02                       48.736001                79.342139   
2012-03                       64.589499                80.184552   
2012-04                       77.144399                54.499909   
2012-05                       79.644357                50.386871   
...                                 ...                      ...   
2023-05                       50.370691                34.107032   
2023-06                       56.978841                49.439589   
2023-07                       28.657913                35.769932   
2023-08                       21.582693                59.098310   
2023-09                       43.435723                55.132444   

            ontario_lake_precipitation  michigan-huron_lake_precipitation  \
year_month                            

In [67]:
# Initialize an empty dataframe to store predictions by model name
model_predictions = []

# Process each model and add to DB
for model_info in models_info:
    model_name = model_info['model']
    df_y = predict_cnbs(X, x_scaler, y_scaler, models_info, model_name)
    if df_y is not None:
        # Store the predictions in the dataframe
        df_y['model'] = model_name
        model_predictions.append(df_y)

df = pd.concat(model_predictions, ignore_index=False).reset_index()

4682/4682 ━━━━━━━━━━━━━━━━━━━━ 1s 187us/step


In [68]:
print(df)

       year_month  superior_evaporation  superior_precipitation  \
0         2012-01             42.779758               57.445095   
1         2012-02             35.585658               24.362964   
2         2012-03             48.398228               35.548518   
3         2012-04             36.972628               56.491740   
4         2012-05             31.295136               71.938297   
...           ...                   ...                     ...   
599267    2023-05             23.957247               48.909592   
599268    2023-06             17.510017               63.761879   
599269    2023-07              4.370193               55.214603   
599270    2023-08              6.096179               41.639763   
599271    2023-09             22.719646               43.476391   

        superior_runoff  erie_evaporation  erie_precipitation  erie_runoff  \
0             24.634651         12.338539           46.373106    95.327450   
1             23.948172         -2.4895

In [69]:
df['superior_cnbs'] = df['superior_precipitation'] - df['superior_evaporation'] + df['superior_runoff']
df['erie_cnbs'] = df['erie_precipitation'] - df['erie_evaporation'] + df['erie_runoff']
df['ontario_cnbs'] = df['ontario_precipitation'] - df['ontario_evaporation'] + df['ontario_runoff']
df['michigan-huron_cnbs'] = df['michigan-huron_precipitation'] - df['michigan-huron_evaporation'] + df['michigan-huron_runoff']

In [70]:
median_df = df.groupby(['model', 'year_month']).median(numeric_only=True).reset_index()

In [ ]:
# Open L2 data 
sup_evap = pd.read_csv(dir + 'l2swbm/superiorEvap_MonthlyRun.csv')
sup_runoff = pd.read_csv(dir + 'l2swbm/superiorRunoff_MonthlyRun.csv')
sup_precip = pd.read_csv(dir + 'l2swbm/superiorPrecip_MonthlyRun.csv')

eri_evap = pd.read_csv(dir + 'l2swbm/erieEvap_MonthlyRun.csv')
eri_runoff = pd.read_csv(dir + 'l2swbm/erieRunoff_MonthlyRun.csv')
eri_precip = pd.read_csv(dir + 'l2swbm/eriePrecip_MonthlyRun.csv')

ont_evap = pd.read_csv(dir + 'l2swbm/ontarioEvap_MonthlyRun.csv')
ont_runoff = pd.read_csv(dir + 'l2swbm/ontarioRunoff_MonthlyRun.csv')
ont_precip = pd.read_csv(dir + 'l2swbm/ontarioPrecip_MonthlyRun.csv')

mih_evap = pd.read_csv(dir + 'l2swbm/miHuronEvap_MonthlyRun.csv')
mih_runoff = pd.read_csv(dir + 'l2swbm/miHuronRunoff_MonthlyRun.csv')
mih_precip = pd.read_csv(dir + 'l2swbm/miHuronPrecip_MonthlyRun.csv')

In [56]:
l2_obs = pd.DataFrame({
    'year_month': pd.to_datetime(eri_evap['Year'].astype(int).astype(str) + '-' + eri_evap['Month'].astype(int).astype(str), format='%Y-%m').dt.strftime('%Y-%m'),
    'superior_evaporation_obs': sup_evap['Median'],
    'superior_precipitation_obs': sup_precip['Median'],
    'superior_runoff_obs': sup_runoff['Median'],
    'erie_evaporation_obs': eri_evap['Median'],
    'erie_precipitation_obs': eri_precip['Median'],
    'erie_runoff_obs': eri_runoff['Median'],
    'ontario_evaporation_obs': ont_evap['Median'],
    'ontario_precipitation_obs': ont_precip['Median'],
    'ontario_runoff_obs': ont_runoff['Median'],
    'michigan-huron_evaporation_obs': mih_evap['Median'],
    'michigan-huron_precipitation_obs': mih_precip['Median'],
    'michigan-huron_runoff_obs': mih_runoff['Median']
})

In [74]:
df_merged = pd.merge(median_df, l2_obs, on=('year_month'), how='left')
df_combined = pd.merge(df_merged, df_obs_merged, on=('year_month'), how='left')
df_combined = df_combined.dropna()

In [75]:
print(df_combined)

    model year_month  superior_evaporation  superior_precipitation  \
0      GP    2012-01             42.213035               33.603582   
1      GP    2012-02             37.574390               23.500649   
2      GP    2012-03             39.481405               38.098477   
3      GP    2012-04             37.600393               39.954112   
4      GP    2012-05             28.184161               55.472621   
..    ...        ...                   ...                     ...   
550    RF    2022-08             13.650148               63.821598   
551    RF    2022-09             20.314967               61.242272   
552    RF    2022-10             39.073391               47.181019   
553    RF    2022-11             47.684511               35.335570   
554    RF    2022-12             52.174866               31.639203   

     superior_runoff  erie_evaporation  erie_precipitation  erie_runoff  \
0          27.953909          9.273942           35.646319    78.555038   
1        

In [78]:
# Save to a CSV [mm]
df.to_csv('CNBS_forecast_archived_obs.csv', sep='\t', index=False)

In [ ]:
df_combined_gp = df_combined[df_combined['model'] == 'GP']
df_combined_lr = df_combined[df_combined['model'] == 'LR']
df_combined_rf = df_combined[df_combined['model'] == 'RF']
df_combined_nn = df_combined[df_combined['model'] == 'NN']

In [83]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from properscoring import crps_ensemble

def calculate_total_skill(df, variables, model, label):
    obs_cols = [v + '_obs' for v in variables]
    
    df = df[df['model'] == model]

    # Combine and drop NaNs
    valid_rows = df[variables + obs_cols].dropna()
    y_pred = valid_rows[variables].values.flatten()
    y_true = valid_rows[obs_cols].values.flatten()

    # Standardize
    scaler = StandardScaler()
    y_true_std = scaler.fit_transform(y_true.reshape(-1, 1)).flatten()
    y_pred_std = scaler.transform(y_pred.reshape(-1, 1)).flatten()

    # Metrics
    rmse = np.sqrt(mean_squared_error(y_true_std, y_pred_std))
    r2 = r2_score(y_true_std, y_pred_std)
    bias = np.mean(y_pred_std - y_true_std)
    variance = np.var(y_pred_std)
    crps = np.mean(crps_ensemble(y_true_std, y_pred_std[:, None]))  # Ensemble shape [n, 1]

    return pd.DataFrame([{
        'group': label,
        'model': model,
        'rmse': round(rmse, 3),
        'r2': round(r2, 3),
        'bias': round(bias, 3),
        'variance': round(variance, 3),
        'crps': round(crps, 3)
    }])

In [85]:
components = ['precipitation', 'evaporation', 'runoff']
lakes = ['erie', 'superior', 'ontario', 'michigan-huron']

# Variable groups
wb_vars = [f'{lake}_{comp}' for lake in lakes for comp in components]
cnbs_vars = [f'{lake}_cnbs' for lake in lakes]

# Calculate total skill
gp_wb_skill = calculate_total_skill(df_combined, wb_vars, 'GP', 'Water Balance Components')
gp_cnbs_skill = calculate_total_skill(df_combined, cnbs_vars, 'GP', 'CNBS')

rf_wb_skill = calculate_total_skill(df_combined, wb_vars, 'RF', 'Water Balance Components')
rf_cnbs_skill = calculate_total_skill(df_combined, cnbs_vars, 'RF', 'CNBS')

lr_wb_skill = calculate_total_skill(df_combined, wb_vars, 'LR', 'Water Balance Components')
lr_cnbs_skill = calculate_total_skill(df_combined, cnbs_vars, 'LR', 'CNBS')

nn_wb_skill = calculate_total_skill(df_combined, wb_vars, 'NN', 'Water Balance Components')
nn_cnbs_skill = calculate_total_skill(df_combined, cnbs_vars, 'NN', 'CNBS')

# Combine results
all_skill = pd.concat([gp_wb_skill, gp_cnbs_skill, rf_wb_skill, rf_cnbs_skill, lr_wb_skill, lr_cnbs_skill, nn_wb_skill, nn_cnbs_skill])
print(all_skill)

                      group model   rmse     r2   bias  variance   crps
0  Water Balance Components    GP  0.867  0.248 -0.407     0.359  0.661
0                      CNBS    GP  1.254 -0.573 -0.762     0.000  0.850
0  Water Balance Components    RF  0.863  0.255 -0.260     0.526  0.639
0                      CNBS    RF  1.251 -0.565 -0.752     0.000  0.846
0  Water Balance Components    LR  0.954  0.090 -0.223     0.823  0.771
0                      CNBS    LR  1.255 -0.575 -0.763     0.001  0.849
0  Water Balance Components    NN  0.896  0.198 -0.389     0.587  0.686
0                      CNBS    NN  1.252 -0.567 -0.757     0.000  0.847
